# Ingest Constructor File
- 1. Read the json file
- 2. change the data types
- 3. Drop the column url using drop
- 4. Rename the col id,requirement
- 5. Add Ingestion Date col
- 6. Write the dataframe to the adls storage in parquet format

In [0]:
dbutils.widgets.help()

In [0]:
dbutils.widgets.text('p_data_source','')
v_data_source = dbutils.widgets.get('p_data_source')

In [0]:
dbutils.widgets.text("p_file_date","2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

### update schema


In [0]:
constructor_schema = "constructorId INT NOT NULL, constructorRef STRING, name STRING, nationality STRING, url STRING"

### read file

In [0]:
display(dbutils.fs.mounts())

In [0]:
display(dbutils.fs.ls(f'{raw_folder_path}'))

In [0]:
display(dbutils.fs.ls(f'{raw_folder_path}/{v_file_date}/constructors.json'))

In [0]:
constructor_read_df = spark.read.schema(constructor_schema).json(f'{raw_folder_path}/{v_file_date}/constructors.json')

In [0]:
display(constructor_read_df)

### drop url

In [0]:
constructor_read_df.printSchema()

In [0]:
constructor_dropped_df = constructor_read_df.drop(constructor_read_df.url)

In [0]:
display(constructor_dropped_df)

In [0]:
display(constructor_dropped_df.describe())

### rename and add

In [0]:
from pyspark.sql.functions import current_timestamp,lit

In [0]:
constructor_renamed_df = constructor_dropped_df\
.withColumnRenamed('constructorId','constructor_id')\
.withColumnRenamed('constructorRef','constructor_ref')\
.withColumn('data_source', lit(v_data_source))\
.withColumn('file_date',lit(v_file_date))

In [0]:
constructor_df = add_ingestion_date(constructor_renamed_df)

In [0]:
display(constructor_df)

In [0]:
constructor_df.write.mode('overwrite').format('parquet').saveAsTable('f1_processed.constructors')

In [0]:
%sql
SELECT * FROM f1_processed.constructors

In [0]:
dbutils.notebook.exit('Success')